## This notebook shows all the means to select some spots

In [ ]:
%matplotlib inline

import time

import torch

from laueimproc import Diagram  # you don't need to import more to access all the filters
from laueimproc.io import get_samples  # gives access to the dataset

### Firstly, we have to iniitialize a diagram with a lot of spots
* For more details about the ways to create a Diagram, please have a look on the `api_init_diagram` notebook.

In [ ]:
file = next(get_samples().iterdir())
diag_ref = Diagram(file)
diag_ref.find_spots()  # see `peaks_search` for details

### Secondly, let filtering!

In [ ]:
"""Very basic slow examples for demonstration."""

# first) Having an idear of the initial diagram
diag1 = diag_ref.clone()
print("*** before filtering ***")
print(diag1)

# 1) Filtering by indices
indices_to_keep = list(range(0, len(diag1), 2))
diag2 = diag1.filter_spots(indices_to_keep, "keep even spots from indices", inplace=False)

# 2) filtering by mask
mask_to_keep = [True if i%2 == 0 else False for i in range(len(diag2))]
diag3 = diag2.filter_spots(mask_to_keep, "keep even spots from mask", inplace=False)

# 3) sorted the spots
criteria = diag3.compute_rois_sum()
sorted_indices = torch.argsort(criteria, descending=True)
diag4 = diag3.filter_spots(sorted_indices, "sorted spots by intensities", inplace=False)

# final) Resume current state
print("*** after filtering ***")
print(diag4)

In [ ]:
"""Same examples with no fork and inplace (more efficient)."""

# first) Having an idear of the initial diagram
diag = diag_ref.clone()
print("*** before filtering ***")
print(diag)

# 1) Filtering by indices
indices_to_keep = torch.arange(0, len(diag), 2)
diag.filter_spots(indices_to_keep, "keep even spots from indices")

# 2) filtering by mask
mask_to_keep = torch.arange(len(diag))%2 == 0
diag.filter_spots(mask_to_keep, "keep even spots from mask")

# 3) sorted the spots
criteria = diag.compute_rois_sum()
sorted_indices = torch.argsort(criteria, descending=True)
diag.filter_spots(sorted_indices, "sorted spots by intensities")

# final) Resume current state
print("*** after filtering ***")
print(diag)

### Thridly, compare speed

In [ ]:
"""Timing the means to filter."""

# exec the cell twice

INDEXS = torch.arange(0, len(diag_ref), 2)
MASK = torch.arange(len(diag_ref)) % 2 == 0


def filter_inplace_index(diag):
    diag.filter_spots(INDEXS, "keep even spots from indices", inplace=True)

def filter_inplace_cond(diag):
    diag.filter_spots(MASK, "keep even spots from mask", inplace=True)

def filter_copy_index(diag):
    diag = diag.filter_spots(INDEXS, "keep even spots from indices", inplace=False)

def filter_copy_cond(diag):
    diag = diag.filter_spots(MASK, "keep even spots from mask", inplace=False)

def timer(func):
    duration = torch.inf
    for _ in range(5):
        diags = [diag_ref.clone() for _ in range(100)]
        t_start = time.time()
        for diag_ in diags:
            func(diag_)
        duration = min(duration, (time.time()-t_start)/len(diag))
    return duration

time_copy_index = timer(filter_copy_index)
time_inplace_index = timer(filter_inplace_index)
time_copy_cond = timer(filter_copy_cond)
time_inplace_cond = timer(filter_inplace_cond)
print(f"inplace (index) is {time_copy_index/time_inplace_index:.2f} times faster than copy")
print(f"inplace (cond) is {time_copy_cond/time_inplace_cond:.2f} times faster than copy")